# Engage in a conversation around structured data using `Granite-Code-20B.`
This notebook showcases querying any dataset, such as social media dataset (Twitter in this case), using natural language to generate SQL queries with the help of LLM (Granite-Code-20B). As we go through this notebook, we will come across two different kinds of prompting techniques incorporated into this demo. One being `zero-shot` prompting, which includes a straightforward prompt, and another one being `few-shot` prompting, where in addition to a prompt, we have given a couple of examples for the model to understand and generate a better SQL query. At the end, we also tested out the `chain of thought` reasoning.

***

## Environment Setup

### Install Granite `utils` package

This package is a thin shim with various functions that are required for notebooks.

To see the implementation of its functions, see the [utils repo](https://github.com/ibm-granite-community/utils/tree/main).

In [ ]:
!pip install git+https://github.com/ibm-granite-community/utils

#### Install all the necessary libraries and packages

In [ ]:
!pip install replicate
!pip install langchain-community
!pip install sqlite-utils
!pip install google-api-python-client

### Find the database file for the dataset

The below code snippet connects to a SQLite database (social_media.db) using the SQLDatabase utility from the langchain_community package and retrieves its schema.

**NOTE** - You can recreate the SQLite database (social_media.db) from CSV files `TwitterDataset/Tweet.csv`, `TwitterDataset/Location.csv`, `TwitterDataset/User.csv` by running Notebook `TwitterDataset/TwitterSyntheticDataGen.ipynb`

In [3]:
from langchain_community.utilities import SQLDatabase

# Note: to run in Colab, you need to upload the TwitterDataset//social_media.db file in the repo to the Colab folder first.
db = SQLDatabase.from_uri("sqlite:///TwitterDataset//social_media.db", sample_rows_in_table_info=0)

def get_schema():
    return db.get_table_info()

Retrieve the SQL schema that will be wrapped in the prompt that we pass to the LLM to have more context and generate accurate response. The schema contains 3 tables:

**location table:** This table stores information about specific locations, including country, state, state code, and city. It is identified by the LocationID column.

**tweet table:** This table contains information about individual tweets, such as the time of posting (Weekday, Hour, Day), language (Lang), whether it was a reshare (IsReshare), tweet statistics (e.g., Reach, RetweetCount, Likes), sentiment, and tweet content (Text). It references two foreign keys: LocationID to the location table and UserID to the user table.

**user table:** This table holds user information, identified by UserID, and stores the user's gender.

In [ ]:
db_schema = get_schema()
print(db_schema)

### Provide your API Token

Obtain your `REPLICATE_API_TOKEN` at [replicate.com/account/api-tokens](https://replicate.com/account/api-tokens)

There are three ways to provide this value to the cells below.  In order of precedence:

1. As an environment variable
2. As a Google colab secret
3. Supplied by the user using `getpass()`

### Choose a Model

Two Granite Code models are available in the [`ibm-granite`](https://replicate.com/ibm-granite) org at Replicate.

The `find_langchain_model` function below imports the `replicate` package.

**Model Parameters**:  
i) max_length: Limits the response length to 100 tokens.  
ii) temperature=0.0: Ensures a deterministic output by minimizing randomness. 

In [5]:
from ibm_granite_community.langchain_utils import find_langchain_model

#model_id = "ibm-granite/granite-8b-code-instruct-128k"
model_id = "ibm-granite/granite-20b-code-instruct-8k"

granite_via_replicate = find_langchain_model(platform="Replicate", model_id=model_id,
                            model_kwargs={"max_length":100, "temperature":0.0})

## Zero-Shot Prompting

This function **zeroshot_prompt** generates a prompt for a language model to create a SQL query based on a natural language question without prior examples (zero-shot prompting).

The prompt is designed as follows:

It positions the model as an experienced social media analyst with expertise in SQL.  
It provides the database schema for context.  
It asks the model to return a SQL query based on the user's question, emphasizing that only the SQL query should be returned.  

In [6]:
def zeroshot_prompt(db_schema, question):
    prompt = f"""You are an Social Media analyst with 15 years of experience writing complex SQL queries.
    Consider the Twitter tables with the following schema:
    {db_schema}

    Write a SQL query that would answer the user's question; just return the SQL query and nothing else.

    Question: {question}

    SQL Query:"""
    return prompt

In the function `get_answer_using_zeroshot()`:  
**granite_via_replicate.invoke(prompt)**: Sends the prompt to the language model to predict the SQL query.     
**result = db.run(sql_query)**: Executes the generated SQL query on the connected social_media.db database.    
**return sql_query, result**: Returns the generated SQL query and the result of the query execution.    

In [7]:
def get_answer_using_zeroshot(db_schema, question):
    prompt = zeroshot_prompt(db_schema, question)
    sql_query = granite_via_replicate.invoke(prompt)
    result = db.run(sql_query)
    
    return sql_query, result

### Testing queries with the Zero-Shot Prompting technique.

In [ ]:
question = "How many tweets are in English?"
sql_query, result = get_answer_using_zeroshot(db_schema, question)
print(f"sql_query : {sql_query}")
print(f"result : {result}")

In [ ]:
question = "Please list the texts of all tweets that are reshared."
sql_query, result = get_answer_using_zeroshot(db_schema, question)
print(f"sql_query : {sql_query}")
print(f"result : {result}")

In [ ]:
question = "How many reshared tweets are there in Ontario?"
sql_query, result = get_answer_using_zeroshot(db_schema, question)
print(f"sql_query : {sql_query}")
print(f"result : {result}")

In [ ]:
question = "Which city has the highest number of tweets?"
sql_query, result = get_answer_using_zeroshot(db_schema, question)
print(f"sql_query: {sql_query}")
print(f"result: {result}")

**NOTE** - For the next question `What is the total number of tweets made by male users on weekdays?`, the LLM couldn't generate the correct SQL query. It generated the wrong condition to filter weekdays: `tweet.Weekday NOT LIKE '%Weekend%'`. This is fixed using the `Few-Shot prompting` in the next section. 

In [ ]:
question = "What is the total number of tweets made by male users on weekdays?"
sql_query, result = get_answer_using_zeroshot(db_schema, question)
print(f"sql_query: {sql_query}")
print(f"result: {result}")

## Few-Shot Prompting

This `fewshot_prompt` function generates a prompt designed for a language model using few-shot prompting, which includes several example SQL queries to guide the model in generating a SQL query from a natural language question.

In [13]:
def fewshot_prompt(db_schema, user_question):
    prompt = f"""
    You are a social media data analyst with 15 years of experience writing complex SQL queries. Generate SQL queries by understanding user input.

    You will be given the entire database which is being queired: {db_schema}
    Your task is to come with the SQL query from the plaintext provided by the user, which when queried on the above database will result in accurate output.
    You are only required generate the SQL query. Do not generate the output from that SQL query or any other explanation.
    Do not generate any explanation or comments at all. Just give the SQL query you came up with as it is.
    Few examples are given below for your reference.

    Example 1: What is the total number of tweets made by female users on weekends?
    Answer Query 1: SELECT COUNT(*) AS total_tweets FROM tweet T JOIN user U ON T.UserID = U.UserID WHERE U.Gender = 'Female' AND T.Weekday IN ('Saturday', 'Sunday');

    Example 2: What is the average sentiment score for tweets made in the United States?
    Answer Query 2: SELECT AVG(T.Sentiment) AS avg_sentiment FROM tweet T JOIN location L ON T.LocationID = L.LocationID WHERE L.Country = 'United States';

    Example 3: List the top 5 users with the highest total reach across all their tweets.
    Answer Query 3: SELECT U.UserID, SUM(T.Reach) AS total_reach FROM tweet T JOIN user U ON T.UserID = U.UserID GROUP BY U.UserID ORDER BY total_reach DESC LIMIT 5;
    User: {user_question}
    Assistant:
    """

    return prompt

In [14]:
def get_answer_using_fewshot(db_schema, question):
    """
    Retrieves the SQL query and result by using few-shot learning for a given database schema and question.
    Args:
        db_schema (str): The database schema.
        question (str): The question to be answered.
    Returns:
        tuple: A tuple containing the SQL query and the result of executing the query on the database.
    """
    prompt = fewshot_prompt(db_schema, question)
    sql_query = granite_via_replicate.invoke(prompt)
    result = db.run(sql_query)
    
    return sql_query, result

### Testing queries with the Few-Shot Prompting technique.

In [ ]:
question = "What is the total number of tweets made by male users on weekdays?"
sql_query, result = get_answer_using_fewshot(db_schema, question)
print(f"sql_query: {sql_query}")
print(f"result: {result}")

**NOTE** - As seen above, the same exact query in zero-shot yielded wrong answer, and here, as we gave some examples to the model, it was able to come up with a correct SQL query for the same user input.  

This goes on to show how having couple of examples related to your domain in the prompt helps out the LLM to accurately generate the desired output.

In [ ]:
question = "Which city has the highest number of tweets?"
sql_query, result = get_answer_using_fewshot(db_schema, question)
print(f"sql_query: {sql_query}")
print(f"result: {result}")

**NOTE** - We have executed the same query in the zero-shot prompting. But the query generated in a few-shot prompting adapted the style of the given examples.

In [ ]:
question = "Find the total reach and average likes for tweets that were reshared by male users."
sql_query, result = get_answer_using_fewshot(db_schema, question)
print(f"sql_query: {sql_query}")
print(f"result: {result}")

In [ ]:
question = "Retrieve the list of users who posted tweets with a sentiment score below 0, grouped by their gender, along with the count of such tweets."
sql_query, result = get_answer_using_fewshot(db_schema, question)
print(f"sql_query: {sql_query}")
print(f"result: {result}")

**NOTE** - For the next question `Find the user(s) who posted the maximum number of tweets in a single day` the LLM couldn't generate the correct SQL query. It missed to include `per day` in the generated SQL query. This is fixed using the `Chain of Thought Reasoning` in the next section. 

In [ ]:
question = "Find the user(s) who posted the maximum number of tweets in a single day."
sql_query, result = get_answer_using_fewshot(db_schema, question)
print(f"sql_query: {sql_query}")
print(f"result: {result}")

## Chain of Thought Reasoning

Chain of Thought (CoT) prompting is used to improve the accuracy of complex reasoning tasks by guiding the model through a structured sequence of steps.  

In this technique, we break down the task into a sequence of logical steps to help the language model understand the process required to generate an accurate SQL query.

In [ ]:
question = """Find the user(s) who posted the maximum number of tweets in a single day.
              First calculate tweets posted by each user on each day.
              Second sort the result in descending order of the number of tweets.
              Third findout the maximum number of tweets posted by a user in a single day.
              Finally select the user(s) who posted the maximum number of tweets in a single day."""

sql_query, result = get_answer_using_fewshot(db_schema, question)
print(f"sql_query: {sql_query}")
print(f"result: {result}")

In [ ]:
question = """Among all tweets with a positive sentiment, what is the percentage of all those posted by a male user?
              First calculate tweets with a positive sentiment those posted by a male user.
              Second calculate all tweets with a positive sentiment.
              Finally calculate the percentage of the first result of the second result."""

sql_query, result = get_answer_using_fewshot(db_schema, question)
print(f"sql_query: {sql_query}")
print(f"result: {result}")